<a href="https://colab.research.google.com/github/ntua-unit-of-control-and-informatics/jaqpot-google-collab-examples/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This example demonstrates a Graph Classification example using Aqueous Solubility dataset.

In [1]:
!pip install PyTDC --quiet
!pip install jaqpotpy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 741.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43

Import required Libraries

In [2]:
import warnings
import torch
from torch_geometric.loader import DataLoader
from jaqpotpy import Jaqpot
from jaqpotpy.descriptors.graph import SmilesGraphFeaturizer
from jaqpotpy.datasets import SmilesGraphDataset
from jaqpotpy.models.torch_geometric_models.graph_neural_network import GraphSageNetwork, pyg_to_onnx
from jaqpotpy.models.trainers.graph_trainers import BinaryGraphModelTrainer

# This is do disable any rdkit warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
warnings.simplefilter("ignore")

Load the regression data. We use TDC library to obtain train, validation, and test splits with SMILES and Solubility.

In [4]:
from tdc.single_pred import Tox
data = Tox(name = 'AMES')
data_splits = data.get_split()

def split_to_list(split):
    return data_splits[split]['Drug'].to_list(), data_splits[split]['Y'].to_list()

# List of smiles and endpoints
train_smiles , train_y =  split_to_list('train')
val_smiles, val_y = split_to_list('valid')
test_smiles , test_y =  split_to_list('test')

Found local copy...
Loading...
Done!


Firstly, a SmilesGraphFeaturizer instance is created. We add 4 features in the featurizer and their specified value (if needed).

In [5]:
featurizer = SmilesGraphFeaturizer(include_edge_features=False)
featurizer.add_atom_feature(
    "symbol",
    ['C', 'K', 'S', 'Br', 'H', 'F', 'N', 'Cl', 'P', 'O', 'I', 'UNK']
)
featurizer.add_atom_feature("total_num_hs", [0, 1, 2, 3, 4])
featurizer.add_atom_feature("degree", [0, 1, 2, 3, 4, 5, 6])
featurizer.add_atom_feature("is_aromatic")

Create train, validation and testing datasets

In [6]:
train_dataset = SmilesGraphDataset(
    smiles=train_smiles, y=train_y, featurizer=featurizer
)
val_dataset = SmilesGraphDataset(
    smiles=val_smiles, y=val_y, featurizer=featurizer
)
test_dataset = SmilesGraphDataset(
    smiles=test_smiles, y=test_y, featurizer=featurizer
)

train_dataset.precompute_featurization()
val_dataset.precompute_featurization()
test_dataset.precompute_featurization()

Create a model. For this demonstration GraphConvolutionNetwork is used with only node features.

In [7]:
node_features = featurizer.get_num_node_features()
model = GraphSageNetwork(
    input_dim=node_features,
    hidden_layers=1,
    hidden_dim=32,
    output_dim=1,
    activation=torch.nn.ReLU(),
    dropout_proba=0.2,
    batch_norm = False,
    seed = 42,
    pooling="add",
)

ReLU()


Define optimizer and loss function

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
loss = torch.nn.BCEWithLogitsLoss()

Create instance of Regression Trainer

In [9]:
trainer = BinaryGraphModelTrainer(
    model=model, # Jaqpotpy Graph Model
    n_epochs=20,
    optimizer=optimizer,
    loss_fn=loss,
    scheduler=None
)

Create PyTorch geometric dataloaders

In [10]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Train and evaluate on the validation set for 20 epochs

In [11]:
trainer.train(train_loader, val_loader)

Epoch 1/20: 100%|██████████| 40/40 [00:00<00:00, 60.59it/s, loss=0.749]


 Train: loss=0.7486 | accuracy=0.6199 | balanced_accuracy=0.6116 | precision=0.6325 | recall=0.7119 | f1=0.6698 | mcc=0.2280 | roc_auc=0.6750


INFO:jaqpotpy.models.trainers.base_trainer: Train: loss=0.7486 | accuracy=0.6199 | balanced_accuracy=0.6116 | precision=0.6325 | recall=0.7119 | f1=0.6698 | mcc=0.2280 | roc_auc=0.6750


 Val:   loss=0.6378 | accuracy=0.6181 | balanced_accuracy=0.6025 | precision=0.6534 | recall=0.7098 | f1=0.6805 | mcc=0.2092 | roc_auc=0.6855


INFO:jaqpotpy.models.trainers.base_trainer: Val:   loss=0.6378 | accuracy=0.6181 | balanced_accuracy=0.6025 | precision=0.6534 | recall=0.7098 | f1=0.6805 | mcc=0.2092 | roc_auc=0.6855
Epoch 2/20: 100%|██████████| 40/40 [00:00<00:00, 60.47it/s, loss=0.658]


...

INFO:jaqpotpy.models.trainers.base_trainer: Val:   loss=0.5465 | accuracy=0.7418 | balanced_accuracy=0.7492 | precision=0.8244 | recall=0.6978 | f1=0.7558 | mcc=0.4934 | roc_auc=0.8136
Epoch 20/20: 100%|██████████| 40/40 [00:01<00:00, 32.32it/s, loss=0.537]


 Train: loss=0.5372 | accuracy=0.7501 | balanced_accuracy=0.7546 | precision=0.8122 | recall=0.7006 | f1=0.7523 | mcc=0.5085 | roc_auc=0.8310


INFO:jaqpotpy.models.trainers.base_trainer: Train: loss=0.5372 | accuracy=0.7501 | balanced_accuracy=0.7546 | precision=0.8122 | recall=0.7006 | f1=0.7523 | mcc=0.5085 | roc_auc=0.8310


 Val:   loss=0.5383 | accuracy=0.7486 | balanced_accuracy=0.7556 | precision=0.8287 | recall=0.7074 | f1=0.7633 | mcc=0.5060 | roc_auc=0.8188


INFO:jaqpotpy.models.trainers.base_trainer: Val:   loss=0.5383 | accuracy=0.7486 | balanced_accuracy=0.7556 | precision=0.8287 | recall=0.7074 | f1=0.7633 | mcc=0.5060 | roc_auc=0.8188


Evaluate and show metrics on the test dataset

In [12]:
loss, metrics, conf_matrix = trainer.evaluate(test_loader)

In [13]:
metrics

{'accuracy': 0.7376373626373627,
 'balanced_accuracy': 0.7437210046392577,
 'precision': 0.8104477611940298,
 'recall': 0.6804511278195489,
 'f1': 0.7397820163487738,
 'mcc': 0.4867306256738066,
 'roc_auc': 0.8269381660839028,
 'loss': 0.5275376163996183}

In [14]:
conf_matrix

array([[531, 127],
       [255, 543]])

Convert PyTorch Geomtric model to onnx format

In [15]:
onnx_model = pyg_to_onnx(model, featurizer)

Log into jaqpot

In [16]:
jaqpot = Jaqpot()
jaqpot.login()

Open this URL in your browser and log in:
https://login.jaqpot.org/realms/jaqpot/protocol/openid-connect/auth?client_id=jaqpot-client&response_type=code&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=openid email profile&state=random_state_value
Enter the authorization code you received: ad4b99b3-d552-4a19-a2ec-5f6ed2c4fc0c.2b052e5d-ef71-411f-bdd8-fb42ad1cbc5e.40e0db1a-58ce-461a-8fbb-6a4451d8587a


Deploy the model on the web

In [ ]:
jaqpot.deploy_torch_model(
    onnx_model,
    featurizer=featurizer,  # Featurizer used for the model
    name="Graph Neural Network",
    description="Graph Sage Network for AMES mutagenicity classification",
    target_name="SOLUBILITY",
    visibility="PRIVATE",
    task="binary_classification",  # Specify the task (regression or binary_classification)
)

  2024-11-12 08:40:22,427 - INFO - Model has been successfully uploaded. The url of the model is https://app.jaqpot.org/dashboard/models/1911
